In [ ]:
import boto3
import pandas as pd
import os
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.parameter import IntegerParameter
from sagemaker.s3 import S3Uploader
from sagemaker.tuner import HyperparameterTuner
from sklearn.model_selection import train_test_split
import requests
import pandas as pd
import json

region = boto3.Session().region_name
sm_client = boto3.client('sagemaker', region_name=region)

In [ ]:
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

In [ ]:
role = 'AmazonSageMaker-ExecutionRole-20210301T231771'
account_id = 487576569445

In [ ]:
role = 'arn:aws:iam::487576569445:role/service-role/AmazonSageMaker-ExecutionRole-20210301T231771'

In [ ]:
model_name = 'single-model-fastapi-1'
model_url = 's3://sagemaker-output-test-1/test-job-11/output/model.tar.gz'
container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, 'test-sagemaker-deploy-container')

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'SingleModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container]
)

print("Model Arn: " + create_model_response['ModelArn'])

In [ ]:
endpoint_config_name = 'DEMO-SignleEndpointConfig-2'
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.large',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [ ]:
import time

endpoint_name = 'DEMO-SingleModelEndpoint-2'
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

In [ ]:
import json
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps({
        'PassengerId': 1,
        'Pclass': 3,
        'Name': 'Braund, Mr. Owen Harris',
        'Sex': 'male',
        'Age': 22.0,
        'SibSp': 1,
        'Parch': 0,
        'Ticket': 'A/5 21171',
        'Fare': 7.25,
        'Cabin': None,
        'Embarked': 'S'})
)
json.loads(response['Body'].read())

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)